In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/final-eval/bart_cobracorpus_predictions.csv


In [5]:
!pip install -q nltk rouge-score bert-score detoxify
import nltk
nltk.download('punkt')


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.8 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-cu12 25.2.0 which is incompatible.


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
%pip install rouge_score
%pip install bert_score
%pip install detoxify

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from detoxify import Detoxify
from tqdm import tqdm

# Load your CSV
df = pd.read_csv("/kaggle/input/final-eval/bart_cobracorpus_predictions.csv")  # Update path if needed

references = df["counterspeech"].astype(str).tolist()
predictions = df["generated_counterspeech"].astype(str).tolist()

# Initialize
smoothie = SmoothingFunction().method4
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

bleu_scores = []
rouge_l_scores = []

# Compute BLEU-1 and ROUGE-L
for ref, pred in tqdm(zip(references, predictions), total=len(references), desc="Scoring"):
    ref_tokens = [ref.split()]
    pred_tokens = pred.split()

    # BLEU-1 only (unigram)
    bleu = sentence_bleu(ref_tokens, pred_tokens, weights=(1, 0, 0, 0), smoothing_function=smoothie)
    rouge_l = rouge.score(ref, pred)['rougeL'].fmeasure

    bleu_scores.append(bleu)
    rouge_l_scores.append(rouge_l)

# Compute BERTScore-F1
_, _, bert_f1 = bert_score(predictions, references, lang="en", verbose=True)
bert_f1 = bert_f1.tolist()

# Compute Toxicity
toxicity = Detoxify('original').predict(predictions)["toxicity"]

# Save final scores
results = {
    "BLEU": sum(bleu_scores) / len(bleu_scores),
    "ROUGE-L": sum(rouge_l_scores) / len(rouge_l_scores),
    "BERTScore-F1": sum(bert_f1) / len(bert_f1),
    "Toxicity": sum(toxicity) / len(toxicity),
}

# Display
results_df = pd.DataFrame([results])
print("\nFinal Evaluation Metrics:")
print(results_df)

# Optional: Save
results_df.to_csv("final_eval_bart_intentcs_bleu1.csv", index=False)


Scoring: 100%|██████████| 2971/2971 [00:04<00:00, 737.91it/s] 


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]


Final Evaluation Metrics:
       BLEU   ROUGE-L  BERTScore-F1  Toxicity
0  0.192042  0.202317      0.877143  0.050323
